In [ ]:
!pip install sweetviz
!pip install plotly
!pip install ydata_profiling
!pip install catboost

ERROR: Ignored the following versions that require a different python version: 4.0.0 Requires-Python >=3.7, <3.11; 4.1.0 Requires-Python >=3.7, <3.12; 4.1.1 Requires-Python >=3.7, <3.12; 4.1.2 Requires-Python >=3.7, <3.12; 4.10.0 Requires-Python <3.13,>=3.7; 4.11.0 Requires-Python <3.13,>=3.7; 4.12.0 Requires-Python <3.13,>=3.7; 4.12.1 Requires-Python <3.13,>=3.7; 4.12.2 Requires-Python <3.13,>=3.7; 4.13.0 Requires-Python <3.13,>=3.7; 4.14.0 Requires-Python <3.13,>=3.7; 4.15.0 Requires-Python <3.13,>=3.7; 4.15.1 Requires-Python <3.13,>=3.7; 4.16.0 Requires-Python <3.13,>=3.7; 4.16.1 Requires-Python <3.13,>=3.7; 4.2.0 Requires-Python >=3.7, <3.12; 4.3.0 Requires-Python >=3.7, <3.12; 4.3.1 Requires-Python >=3.7, <3.12; 4.3.2 Requires-Python >=3.7, <3.12; 4.4.0 Requires-Python >=3.7, <3.12; 4.5.0 Requires-Python >=3.7, <3.12; 4.5.1 Requires-Python >=3.7, <3.12; 4.6.0 Requires-Python >=3.7, <3.12; 4.6.1 Requires-Python >=3.7, <3.12; 4.6.2 Requires-Python >=3.7, <3.12; 4.6.3 Requires-Python

In [1]:
# Importar librerías
## EDA
import pandas as pd
import numpy as np
import sweetviz as sv
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Modelamiento
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from catboost import CatBoostClassifier
from scipy.stats import randint, uniform

c:\Users\David Peña\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\David Peña\AppData\Local\Programs\Python\Python313\Lib\site-packages\sweetviz\graph.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


ModuleNotFoundError: No module named 'ydata_profiling'

# a. Análisis Exploratorio

In [ ]:
# Cargar datos
clientes = pd.read_excel('/content/drive/MyDrive/DSA/clientes_final.xlsx')
ventas = pd.read_excel('/content/drive/MyDrive/DSA/venta_final.xlsx')

## Variables

In [ ]:
# Exploración inicial
print("\nClientes DataFrame Info:")
print(clientes.info())

print("\nVentas DataFrame Info:")
print(ventas.info())

# Estadísticas descriptivas
print("\nEstadísticas descriptivas - Clientes:")
print(clientes.describe())

print("\nEstadísticas descriptivas - Ventas:")

In [ ]:
print(ventas.drop_duplicates().shape)
print(clientes.drop_duplicates().shape)

Se asume que los registros deben ser únicos en cada base, por lo tanto, que tanto en la base de clientes como en la de ventas solo existe un registro por cliente y por venta (con la misma información)

In [ ]:
ventas.drop_duplicates(inplace=True)
clientes.drop_duplicates(inplace=True)

## Clientes

In [ ]:
ventas['porc_desc'] = abs(ventas['desc'])/(abs(ventas['desc'])+ ventas['nr'])
ventas['porc_desc_sin_0'] = np.where(ventas['porc_desc'] == 0, np.nan, ventas['porc_desc'])

In [ ]:
ventas['vol_cat'] = np.where((ventas['vol'] > 1)&(ventas['vol'] <= 1.5), '(1-1.5]',
                           np.where((ventas['vol'] > 1.5)&(ventas['vol'] <= 2.5), '(1.5-2.5]',
                                    np.where((ventas['vol'] > 2.5)&(ventas['vol'] <= 9.5), '(2.5-9.5]', '>9.5')))
ventas['nr_cat'] = np.where((ventas['nr'] == 0), '0',
                           np.where((ventas['nr'] > 0)&(ventas['nr'] <= 2600000), '(0-2.6M]',
                                    np.where((ventas['nr'] > 2600000)&(ventas['nr'] <= 3500000), '(2.6M-3.5M]',
                                             np.where((ventas['nr'] > 3500000)&(ventas['nr'] <= 34000000), '(3.5M-34M]', '>34M'))))
ventas['porc_desc_cat'] = np.where((ventas['porc_desc'] == 0), '0%',
                                 np.where((ventas['porc_desc'] > 0)&(ventas['porc_desc'] <= 0.2), '(0%-20%]',
                                          np.where((ventas['porc_desc'] > 0.2)&(ventas['porc_desc'] <= 0.4), '(20%-40%]',
                                                   np.where((ventas['porc_desc'] > 0.4)&(ventas['porc_desc'] <= 0.6), '(40%-60%]',
                                                            np.where((ventas['porc_desc'] > 0.6)&(ventas['porc_desc'] <= 0.8), '(60%-80%]',
                                                                     np.where((ventas['porc_desc'] > 0.8)&(ventas['porc_desc'] <= 1), '(80%-100%]', 'No_Aplica'))))))

In [ ]:
abt = pd.merge(ventas, clientes, on='Cliente', how='left')

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
abt.groupby(['Gerencia', 'subcanal']).agg({'Cliente': 'nunique',
                                           'porc_desc': ['min', 'max', 'mean'],
                                           'nr': ['min', 'max', 'mean', 'sum'],
                                           'vol': ['min', 'max', 'mean', 'sum']})

## Insights

In [ ]:
report =  sv.analyze(abt)
report.show_html('EDA_Fuentes.html')

In [ ]:
profile = ProfileReport(abt, title="EDA Fuentes Clientes & Ventas", minimal=True)
profile.to_file("EDA_Fuentes_2.html")

In [ ]:
# Gráfico de dispersión productos vendidos vs. ganancia
fig = px.scatter(abt, x='vol', y='nr', color='porc_desc_cat',
                 color_continuous_scale='viridis',
                 title='Gráfica de Cantidad de productos vendidos vs. Ganancia, por grupo de descuento',
                 labels={'vol': 'Cantidad de productos vendidos',
                         'nr': 'Ganancia',
                         'porc_desc_cat': 'Porcentaje de descuento'})

fig.show()

In [ ]:
abt.groupby(['Cliente', 'ym']).agg({'brand': 'count'}) #Hay más de un registro por cliente y mes

In [ ]:
suma_nr = abt.groupby(['brand'])['nr'].sum()
suma_vol = abt.groupby(['brand'])['vol'].sum()
ganan_x_und = pd.DataFrame({
    'suma_nr': suma_nr,
    'suma_vol': suma_vol
 })
ganan_x_und['ganancia_x_und'] = ganan_x_und['suma_nr'] / ganan_x_und['suma_vol']

In [ ]:
# Proporción de ventas por marca
pd.concat([(100*abt.groupby(['brand']).agg({'Cliente': 'count'})/len(abt)).sort_values(by='Cliente'),
           (abt.groupby(['brand']).agg({'Cliente': 'count'})),
           ganan_x_und['ganancia_x_und']], axis=1)

In [ ]:
# Proporción de ventas por segmento
pd.concat([(100*abt.groupby(['segment']).agg({'Cliente': 'count'})/len(abt)).sort_values(by='Cliente'),
           (abt.groupby(['segment']).agg({'Cliente': 'count'})).sort_values(by='Cliente')], axis=1)

# b. Modelo de venta - Marca1

In [ ]:
# Marca elegida: Marca1
abt_final = abt[['Gerencia', 'subcanal', 'desc', 'brand', 'ym', 'Cliente', 'vol']]
abt_final['target'] = np.where(abt['brand'] == 'Marca1', 1, 0)
abt_final

In [ ]:
# Creación de variable de volumen vendido anteriormente al cliente de esta misma marca
abt_final['ym'] = pd.to_datetime(abt_final['ym'], format='%Y%m')
df_marca1 = abt_final[abt_final['brand'] == 'Marca1'].copy()
df_marca1 = df_marca1.sort_values(by=['Cliente', 'ym'])
df_marca1['vol_vendido_antes_m1'] = df_marca1.groupby('Cliente')['vol'].cumsum() - df_marca1['vol']
result = abt_final.merge(df_marca1[['Cliente', 'ym', 'vol_vendido_antes_m1']],
                         on=['Cliente', 'ym'],
                         how='left')
result['vol_vendido_antes_m1'] = result['vol_vendido_antes_m1'].fillna(0)

# Creación del descuento dado al cliente en los periodos anteriores
df_desc = result.loc[result['brand'] == 'Marca1']
df_desc['abs_desc'] = abs(df_desc['desc'])
df_desc = df_desc.sort_values(by=['Cliente', 'ym'])
df_desc['desc_antes_m1'] = df_desc.groupby('Cliente')['abs_desc'].cumsum() - df_desc['abs_desc']
result2 = result.merge(df_desc[['Cliente', 'ym', 'desc_antes_m1']],
                         on=['Cliente', 'ym'],
                         how='left')
result2['desc_antes_m1'] = result2['desc_antes_m1'].fillna(0)

Se agrupan las ventas por cliente y fecha, para modelar si un cliente comprará en un mes la marca 1

In [ ]:
abt_modelo = result2.groupby(['Cliente', 'ym', 'Gerencia', 'subcanal']).agg({'desc_antes_m1': 'max',
                                        'vol_vendido_antes_m1' : 'max',
                                        'target' : 'max'}).reset_index(drop=False).drop(columns=['Cliente',
                                                                                                 'ym'])
abt_modelo.describe(include='all')

In [ ]:
X = abt_modelo[['desc_antes_m1', 'vol_vendido_antes_m1', 'Gerencia', 'subcanal']]
y = abt_modelo['target']
numeric_features = ['desc_antes_m1', 'vol_vendido_antes_m1']
categorical_features = ['Gerencia', 'subcanal']

numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

X_processed =preprocessor.fit_transform(X)

### Regresión Logística

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=0)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Obtener probabilidades de predicción
y_probs = model.predict_proba(X_test)[:, 1]

# Definir una serie de umbrales
thresholds = np.arange(0.0, 1.1, 0.1)

# Inicializar listas para almacenar métricas
precisions = []
recalls = []
f1_scores = []

# Evaluar el modelo para cada umbral
for threshold in thresholds:
    y_pred = (y_probs >= threshold).astype(int)

    # Calcular métricas
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)

# Graficar resultados
plt.figure(figsize=(5, 3))
plt.plot(thresholds, precisions, label='Precision', marker='o')
plt.plot(thresholds, recalls, label='Recall', marker='o')
plt.plot(thresholds, f1_scores, label='F1 Score', marker='o')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Threshold Analysis')
plt.legend()
plt.grid(True)
plt.show()

# Graficar la curva ROC
fpr, tpr, _ = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(5, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
for threshold in [0.7, 0.8, 0.9]:
  y_probs = model.predict_proba(X_test)[:, 1]
  y_pred = (y_probs >= threshold).astype(int)
  conf_matrix = confusion_matrix(y_test, y_pred)
  print(conf_matrix)
  print(classification_report(y_test, y_pred))

### Importancia de variables

In [ ]:
feature_names = (numeric_features +
                 list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))
coef = model.coef_[0]
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coef
}).sort_values(by='Coefficient', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

## Catboost

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ],
    remainder='passthrough'
)
X_processed = preprocessor.fit_transform(X[numeric_features])
X_categorical = X[categorical_features]
X_final = np.hstack((X_processed, X_categorical))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=0)

# Ajustar el modelo CatBoost
model = CatBoostClassifier(cat_features=[2, 3])
model.fit(X_train, y_train, cat_features=[2, 3], verbose=200)

# Predicciones y evaluación
y_pred = model.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

### Importancia de variables

In [ ]:
feature_importances = model.get_feature_importance()
feature_names = (numeric_features +
                 list(X_categorical.columns))

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(importance_df)

plt.figure(figsize=(5, 3))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Importancia de las Características')
plt.show()

## 🎯 Maqueta del Prototipo

La presente maqueta representa un prototipo analítico orientado a la visualización y comprensión de los datos de ventas de Bavaria. Este prototipo está diseñado para responder preguntas estratégicas como:

- ¿Qué productos generan mayor volumen y rentabilidad?
- ¿Qué segmentos presentan mayores descuentos y cómo se comportan?
- ¿Cómo se distribuyen las ventas por categoría y región?

### Componentes visuales clave:

- **Gráfico de dispersión**: Volumen vs. rentabilidad (`plotly.express`) para identificar productos estrella
- **Mapas de calor y gráficos de barras**: Tendencias por categorías, precios y promociones
- **Perfiles automáticos de datos**: Sweetviz y YData Profiling para análisis exploratorio automático

Esta maqueta servirá como base para desarrollar un dashboard más completo e interactivo, o un informe automatizado para el equipo de decisiones de negocio.
